# Tutorial 7 - Reflection and Self-Correction (Worker + Critic)

## Where You Are in the Learning Journey

```
 Tutorials 1-5      Tutorial 6         Tutorial 7         Tutorial 8
 RAG Fundamentals   ReAct Agent        Reflection         State
 (the retrieval     (T6)               (you are here)     Management
  pipeline)                                               (T8)
```

**What changed from Tutorial 6:** a second agent -- the **Critic** -- is added.
After the Worker produces an answer, the Critic reviews it and either approves
it or sends specific feedback back to the Worker for revision.

**What you will learn in this tutorial:**
- What the Worker-Critic (reflection) pattern is and why it improves answer quality
- How to separate generation from evaluation using two different agent roles
- How to trace multi-round revision cycles
- When reflection helps and when it adds unnecessary overhead

**Prerequisites:** Tutorial 6 (understand the basic ReAct loop). Python basics.

```mermaid
flowchart LR
    Q[Question + Context] --> W[Worker: draft answer]
    W --> C[Critic: review]
    C -- Approved --> F[Final Answer]
    C -- Feedback --> W
```


## Why Does a Single-Pass Answer Need a Critic?

In Tutorial 6, the ReAct agent calls the retriever and generates an answer in
one pass. The agent does not check whether its own answer is correct.

### Problem: Confident Wrong Answers

Language models can produce answers that sound convincing but are factually wrong
or incomplete. Common failure modes:

| Failure Mode | Example |
|--------------|--------|
| Hallucination | States 'employees get 30 days' when context says 25 |
| Missing citation | Answers without saying which policy section it came from |
| Vague answer | Says 'some days are allowed' instead of '14 days maximum' |
| Ignored condition | Omits 'beyond 14 days requires Global Mobility approval' |

### Solution: A Separate Critic Agent

The Critic is a second LLM call with a different system prompt. It reads:
- The original question
- The retrieved context
- The Worker's draft answer

The Critic checks whether the answer is accurate, complete, and grounded in the
context. If not, it returns structured feedback: exactly what is wrong and what
the Worker should fix.

### Why Two Separate Agents Rather Than One?

A single agent asked to 'critique your own answer' tends to agree with itself.
Using a separate Critic call with a different system prompt (focused entirely on
finding problems) produces more reliable error detection.

```
Worker system prompt  : 'Answer the question using only the provided context.'
Critic system prompt  : 'Find errors, missing information, and unsupported claims.'
```


## How the Reflection Loop Works (Round by Round)

```
Round 1:
  Worker receives: question + context
  Worker produces: draft answer v1
  Critic receives: question + context + draft answer v1
  Critic produces: {approved: false, feedback: 'Missing the 14-day limit.'}

Round 2:
  Worker receives: question + context + Critic feedback
  Worker produces: draft answer v2 (with the 14-day limit included)
  Critic receives: question + context + draft answer v2
  Critic produces: {approved: true, feedback: ''}

Result: draft answer v2 is returned as the final answer.
```

The loop continues until either:
- The Critic approves (`approved: true`), or
- `max_rounds` is reached (the last Worker answer is returned as-is)


In [ ]:
import importlib
import os
from pathlib import Path
import shutil
import subprocess
import sys

import pandas as pd
from dotenv import load_dotenv

if shutil.which("uv") is None:
    print("uv not found. Installing with pip...")
    subprocess.run([sys.executable, "-m", "pip", "install", "uv"], check=True)

cwd = Path.cwd().resolve()
repo_root = next(
    (path for path in [cwd, *cwd.parents] if (path / "pyproject.toml").exists() and (path / "src").exists()),
    cwd,
)
os.chdir(repo_root)
src_path = repo_root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

REQUIRED_PACKAGES = ["openai", "chromadb", "numpy", "pandas", "rank_bm25", "sentence_transformers", "dotenv"]
PIP_NAME_MAP = {"rank_bm25": "rank-bm25", "sentence_transformers": "sentence-transformers", "dotenv": "python-dotenv"}

def find_missing(packages):
    importlib.invalidate_caches()
    return [pkg for pkg in packages if importlib.util.find_spec(pkg) is None]

missing = find_missing(REQUIRED_PACKAGES)
if missing:
    print("Missing packages:", missing)
    subprocess.run(["uv", "sync"], check=True)

missing_after_sync = find_missing(REQUIRED_PACKAGES)
if missing_after_sync:
    pip_targets = [PIP_NAME_MAP.get(pkg, pkg) for pkg in missing_after_sync]
    subprocess.run([sys.executable, "-m", "pip", "install", *pip_targets], check=True)

final_missing = find_missing(REQUIRED_PACKAGES)
if final_missing:
    raise ImportError(f"Dependencies still missing: {final_missing}")

from rag_tutorials.io_utils import load_handbook_documents, load_queries
from rag_tutorials.chunking import semantic_chunk_documents
from rag_tutorials.pipeline import build_dense_retriever
from rag_tutorials.qa import answer_with_context

load_dotenv()
if not os.getenv("OPENAI_API_KEY"):
    raise EnvironmentError("OPENAI_API_KEY is required")

embedding_model = os.getenv("OPENAI_EMBEDDING_MODEL", "text-embedding-3-small")
chat_model = os.getenv("OPENAI_CHAT_MODEL", "gpt-4.1-mini")

handbook_path = Path("data/handbook_manual.txt")
queries_path = Path("data/queries.jsonl")
if not handbook_path.exists() or not queries_path.exists():
    raise FileNotFoundError("Run: uv run python scripts/generate_data.py")

documents = load_handbook_documents(handbook_path)
queries = load_queries(queries_path)
chunks = semantic_chunk_documents(documents)
dense_retriever, _ = build_dense_retriever(
    chunks=chunks,
    collection_name="agent_tutorial_dense",
    embedding_model=embedding_model,
)

In [ ]:
# Build the context string that both Worker and Critic will see
# We use the same dense retriever from Tutorials 1-4

from rag_tutorials.reflection import (
    worker_answer,
    critic_review,
    run_reflection_loop,
    CriticFeedback,
)

TOP_K = 5

def retrieve_context(question: str, top_k: int = TOP_K) -> str:
    """Retrieve top-k chunks and join them as a single context string."""
    results = dense_retriever(question, top_k=top_k)
    parts = [f"Chunk {i+1} [{r.chunk_id}]: {r.text}" for i, r in enumerate(results)]
    return "\n\n".join(parts)

print("Context helper ready.")
print("Sample context for 'international work days':")
print(retrieve_context("international work days")[:400], "...")

## Novice Trace: Watching the Worker-Critic Loop

Let us trace a single question through the full reflection loop and print
each round so you can see exactly what changes between revisions.

**Question:** 'How many days can an employee work from another country, and
what must they do if they want to stay longer?'

Watch for:
- Round 1 answer: what does the Worker produce first?
- Critic feedback: what specific problem does the Critic identify?
- Round 2 answer: how does the Worker incorporate the feedback?


In [ ]:
# Single-question reflection trace

question = (
    "How many days can an employee work from another country, and "
    "what must they do if they want to stay longer?"
)
context = retrieve_context(question)

result = run_reflection_loop(
    question=question,
    context=context,
    model=chat_model,
    max_rounds=3,
)

print("QUESTION:", result.question)
print("CONTEXT (first 300 chars):", context[:300], "...")
print("=" * 70)

for entry in result.history:
    print(f"\n--- Round {entry['round']} ---")
    print(f"Worker answer : {entry['answer'][:300]}" + ("..." if len(entry['answer']) > 300 else ""))
    print(f"Critic approved : {entry['approved']}")
    if entry['feedback']:
        print(f"Critic feedback : {entry['feedback']}")

print("\n" + "=" * 70)
print("FINAL ANSWER (after", result.rounds, "rounds):")
print(result.final_answer)

In [ ]:
# Run reflection loop on first 5 queries and compare single-pass vs. multi-round

import time
from rag_tutorials.qa import answer_with_context
from rag_tutorials.evaluation import groundedness_score

rows = []
eval_queries = queries[:5]

for q in eval_queries:
    ctx = retrieve_context(q.question)
    ctx_chunks = [r.text for r in dense_retriever(q.question, top_k=5)]

    # Single-pass answer (no critic)
    t0 = time.perf_counter()
    single_answer = answer_with_context(q.question, ctx_chunks, model=chat_model)
    single_ms = (time.perf_counter() - t0) * 1000

    # Reflection loop
    t1 = time.perf_counter()
    reflection_result = run_reflection_loop(q.question, ctx, model=chat_model, max_rounds=3)
    reflect_ms = (time.perf_counter() - t1) * 1000

    single_g = groundedness_score(single_answer, ctx_chunks)
    reflect_g = groundedness_score(reflection_result.final_answer, ctx_chunks)

    rows.append({
        "query_id": q.query_id,
        "rounds": reflection_result.rounds,
        "single_groundedness": round(single_g, 3),
        "reflect_groundedness": round(reflect_g, 3),
        "single_ms": round(single_ms, 0),
        "reflect_ms": round(reflect_ms, 0),
    })

df = pd.DataFrame(rows)
print(df.to_string(index=False))
print(f"\nMean single groundedness  : {df['single_groundedness'].mean():.3f}")
print(f"Mean reflect groundedness : {df['reflect_groundedness'].mean():.3f}")
print(f"Mean single latency       : {df['single_ms'].mean():.0f} ms")
print(f"Mean reflect latency      : {df['reflect_ms'].mean():.0f} ms")

## Learning Checkpoint: Reflection and Self-Correction

### What Works

- The Critic catches specific problems (missing conditions, vague answers,
  unsupported claims) and produces actionable feedback.
- The Worker incorporates feedback and produces an improved answer in the next
  round.
- Groundedness typically improves because the Critic pushes the Worker to stay
  closer to the retrieved context.

### What Does Not Work Well

- Adding a Critic at least doubles latency (two LLM calls per round, times the
  number of rounds).
- If the retrieved context is wrong or missing, neither Worker nor Critic can
  recover: the Critic can only check against the provided context.
- Multi-round runs are stateless: if something goes wrong mid-run, there is no
  way to pause, inspect, or rewind to an earlier state.

### Why Move to Tutorial 8?

Tutorial 8 adds **state management** with checkpoints and time travel. You will
learn how to save agent state at each step, pause for human review, and rewind
to any previous checkpoint if something goes wrong.
